<a href="https://colab.research.google.com/github/nnilayy/SmolGPT/blob/main/00_mdna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up

In [2]:
!pip install sec-parser tiktoken jupyter_black sec_downloader

In [ ]:
# !pip install tiktoken jupyter_black sec_downloader

In [3]:
from __future__ import annotations
import sec_parser as sp
import sec_downloader as sd
import jupyter_black
import tiktoken

jupyter_black.load()

## Ticker

In [207]:
ticker="CSCO"

## Ticker Data

In [208]:
# Download HTML
dl = sd.Downloader("alphanome.ai", "info@alphanome.ai")
html = dl.get_latest_html("10-Q", ticker)

In [209]:
# Parse HTML
elements = sp.Edgar10QParser().parse(html)
top_level_sections = [
    item for part in sp.TreeBuilder().build(elements) for item in part.children
]

In [210]:
# Filter MD&A section
mdna_top_level_sections = [
    k for k in top_level_sections if "management" in k.semantic_element.text.lower()
]
assert len(mdna_top_level_sections) == 1
mdna_top_level_section = mdna_top_level_sections[0]

In [211]:
# Convert to markdown (Step 1: Get levels)
levels = sorted(
    {
        k.semantic_element.level
        for k in mdna_top_level_section.get_descendants()
        if isinstance(k.semantic_element, sp.TitleElement)
    }
)
level_to_markdown = {level: "#" * (i + 2) for i, level in enumerate(levels)}
level_to_markdown

{0: '##',
 1: '###',
 2: '####',
 4: '#####',
 6: '######',
 7: '#######',
 8: '########'}

In [212]:
# Convert to markdown (Step 2: Extract text)
markdown = ""
markdown += f"# {mdna_top_level_section.semantic_element.text}\n"
for node in mdna_top_level_section.get_descendants():
    element = node.semantic_element
    if isinstance(element, sp.TextElement):
        markdown += f"{element.text}\n"
    elif isinstance(element, sp.TitleElement):
        markdown += f"{level_to_markdown[element.level]} {element.text}\n"
    elif isinstance(element, sp.TableElement):
        markdown += f"[{element.get_summary()}]\n"

In [213]:
print(markdown)

# Item 2.Management’s Discussion and Analysis of Financial Condition and Results of Operations
#### Forward-Looking Statements
This Quarterly Report on Form 10-Q, including this Management’s Discussion and Analysis of Financial Condition and Results of Operations, contains forward-looking statements regarding future events and our future results that are subject to the safe harbors created under the Securities Act of 1933, as amended (the “Securities Act”) and the Securities Exchange Act of 1934, as amended (the “Exchange Act”). All statements other than statements of historical facts are statements that could be deemed forward-looking statements. These statements are based on current expectations, estimates, forecasts, and projections about the industries in which we operate and the beliefs and assumptions of our management. Words such as “expects,” “anticipates,” “targets,” “goals,” “projects,” “intends,” “plans,” “believes,” “momentum,” “seeks,” “estimates,” “continues,” “endeavors,

## Tables

In [214]:
from sec_parser.semantic_elements.table_element.table_element import TableElement

mdna_tables = []

for section in mdna_top_level_sections:
    for descendant in section.get_descendants():
        if isinstance(descendant.semantic_element, TableElement):
            mdna_tables.append(descendant.semantic_element)

## Output

In [215]:
import re

# Your code to find matches
pattern = r"\[Table with \d+ rows, \d+ numbers, and \d+ characters.]"
matches = re.findall(pattern, markdown)
for i, match in zip(range(len(matches)),matches):
    print(str(i)+" "+match)

0 [Table with 15 rows, 91 numbers, and 1143 characters.]
1 [Table with 3 rows, 8 numbers, and 155 characters.]
2 [Table with 3 rows, 8 numbers, and 188 characters.]
3 [Table with 6 rows, 36 numbers, and 428 characters.]
4 [Table with 8 rows, 48 numbers, and 516 characters.]
5 [Table with 8 rows, 48 numbers, and 546 characters.]
6 [Table with 8 rows, 58 numbers, and 573 characters.]
7 [Table with 8 rows, 48 numbers, and 510 characters.]
8 [Table with 4 rows, 32 numbers, and 360 characters.]
9 [Table with 6 rows, 15 numbers, and 214 characters.]
10 [Table with 7 rows, 53 numbers, and 523 characters.]
11 [Table with 8 rows, 52 numbers, and 581 characters.]
12 [Table with 4 rows, 16 numbers, and 220 characters.]
13 [Table with 2 rows, 12 numbers, and 200 characters.]
14 [Table with 3 rows, 22 numbers, and 268 characters.]
15 [Table with 7 rows, 40 numbers, and 488 characters.]
16 [Table with 4 rows, 14 numbers, and 210 characters.]
17 [Table with 3 rows, 8 numbers, and 174 characters.]
18 

In [ ]:
from bs4 import BeautifulSoup
html=mdna_tables[index].html_tag.get_source_code()
soup=BeautifulSoup(html,'html.parser')
soup

In [240]:
from sec_parser.semantic_elements.table_element.table_parser import TableParser
index=1
table=mdna_tables[index].html_tag.get_source_code()
# df=TableParser(table).parse_as_df()
# df

In [291]:
import pandas as pd
from io import StringIO

def basic_preprocessing(html: str) -> pd.DataFrame:
  tables = pd.read_html(StringIO(html), flavor="lxml")
  if len(tables) == 0:
      msg = "No tables found"
      raise ValueError(msg)
  table = tables[0]
  table = table.dropna(how="all")
  table.columns = pd.Index(table.iloc[0].tolist())
  table = table[1:]
  table = table.dropna(axis=1, how="all")
  first_row = list(table.columns)
  new_df = pd.DataFrame([first_row], columns=table.columns)
  table = pd.concat([new_df, table], ignore_index=True)
  table.columns = pd.Index(
      list(range(1, len(table.columns) + 1)),
      dtype=str,
  )
  table = table.fillna("")
  return table.astype(str)

def remove_blank_columns(df: pd.DataFrame) -> pd.DataFrame:
    columns_to_remove = []
    for i, _ in enumerate(df.columns):
        if i < len(df.columns) - 1:
            current_column = df.columns[i]
            next_column = df.columns[i + 1]
            non_empty_rows = df[
                (df[current_column] != "") & (df[next_column] != "")
            ]
            if (
                non_empty_rows[current_column] == non_empty_rows[next_column]
            ).all():
                if (df[current_column] == "").sum() >= (
                    df[next_column] == ""
                ).sum():
                    columns_to_remove.append(current_column)
                else:
                    columns_to_remove.append(next_column)
    return df.drop(columns=columns_to_remove)



# def merge_columns_by_marker(table: pd.DataFrame, marker: str) -> pd.DataFrame:
#     marker_columns = [col for col in table.columns if any(str(value) == marker for value in table[col])]

#     for current_column in marker_columns:
#         current_column_index = table.columns.get_loc(current_column)

#         if current_column_index < len(table.columns) - 1:
#             next_column = table.columns[current_column_index + 1]

#             non_marker_rows = table[(table[current_column] != marker) & (table[next_column] != marker)]
#             non_empty_non_marker_rows = non_marker_rows[(non_marker_rows[current_column] != "") & (non_marker_rows[next_column] != "")]
#             marker_rows = table.drop(non_empty_non_marker_rows.index)
#             marker_rows_indices = marker_rows.index

#             if (non_empty_non_marker_rows[current_column] == non_empty_non_marker_rows[next_column]).all():
#                 table.loc[marker_rows_indices, current_column] = (table.loc[marker_rows_indices, current_column].str.cat(table.loc[marker_rows_indices, next_column], sep="", na_rep="").str.strip())
#                 table = table.drop(columns=[next_column])

#     return table



# def merge_columns_by_marker(table: pd.DataFrame, merge_marker: str) -> pd.DataFrame:
#     marker_columns = [col for col in table.columns if any(str(value) == merge_marker for value in table[col])]

#     for current_column in marker_columns:
#         current_column_index = table.columns.get_loc(current_column)

#         if current_column_index > 0:  # Check if current column is not the first column
#             previous_column = table.columns[current_column_index - 1]

#             non_marker_rows = table[(table[current_column] != merge_marker) & (table[previous_column] != merge_marker)]
#             non_empty_non_marker_rows = non_marker_rows[(non_marker_rows[current_column] != "") & (non_marker_rows[previous_column] != "")]
#             marker_rows = table.drop(non_empty_non_marker_rows.index)
#             marker_rows_indices = marker_rows.index

#             if (non_empty_non_marker_rows[current_column] == non_empty_non_marker_rows[previous_column]).all():
#                 table.loc[marker_rows_indices, previous_column] = (table.loc[marker_rows_indices, previous_column].str.cat(table.loc[marker_rows_indices, current_column], sep="", na_rep="").str.strip())
#                 table = table.drop(columns=[current_column])

#     return table


import pandas as pd

def merge_columns_by_markers(table: pd.DataFrame, markers: list) -> pd.DataFrame:
    for marker in markers:
        marker_columns = [col for col in table.columns if any(str(value) == marker for value in table[col])]

        for current_column in marker_columns:
            current_column_index = table.columns.get_loc(current_column)

            if marker == '$' and current_column_index < len(table.columns) - 1:
                neighbor_column = table.columns[current_column_index + 1]
                left_column=current_column
                right_column=neighbor_column

            elif (marker == '%' or marker == ')') and current_column_index > 0:
                neighbor_column = table.columns[current_column_index - 1]
                left_column=neighbor_column
                right_column=current_column
            else:
                continue  # Skip if no neighbor column in the specified direction

            non_marker_rows = table[(table[current_column] != marker) & (table[neighbor_column] != marker)]
            non_empty_non_marker_rows = non_marker_rows[(non_marker_rows[current_column] != "") & (non_marker_rows[neighbor_column] != "")]
            marker_rows = table.drop(non_empty_non_marker_rows.index)
            marker_rows_indices = marker_rows.index

            if (non_empty_non_marker_rows[current_column] == non_empty_non_marker_rows[neighbor_column]).all():
                table.loc[marker_rows_indices, current_column] = (table.loc[marker_rows_indices, current_column].str.cat(table.loc[marker_rows_indices, neighbor_column], sep="", na_rep="").str.strip())
                table = table.drop(columns=[neighbor_column])

    return table

In [295]:
%%timeit
based_df=basic_preprocessing(table)
based_df=remove_blank_columns(based_df)
based_df=merge_columns_by_markers(based_df,["$","%",")"])
based_df

31.9 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [297]:
table=TABLE_10Q_CP_0000016875_20_000032__001
based_df=basic_preprocessing(table)
based_df=remove_blank_columns(based_df)
based_df=merge_columns_by_markers(based_df,["$","%",")"])
based_df

,1,4,7,10,13
0,,For the three months ended September 30,For the three months ended September 30,For the nine months ended September 30,For the nine months ended September 30
1,(in millions of Canadian dollars),2020,2019,2020,2019
2,Net income,$598,$618,$1642,$1776
3,Net gain (loss) in foreign currency translatio...,16,)(8,)(18,23
4,Change in derivatives designated as cash flow ...,3,2,6,8
5,Change in pension and post-retirement defined ...,44,20,134,61
6,Other comprehensive income before income taxes,63,14,122,92
7,Income tax (expense) recovery on above items,)(29,3,)(16,)(41
8,Other comprehensive income (Note 7),34,17,106,51
9,Comprehensive income,$632,$635,$1748,$1827


In [ ]:
import pandas as pd

def assign_same_value(df):
    first_column = df.columns[0]
    for index, row in df.iterrows():
        if row[first_column] == '' and all(value == row.iloc[1] for value in row.iloc[1:]):
            value_to_assign = row.iloc[1]
            df.loc[index, first_column] = value_to_assign
    return df


def clean_repeating_rows(df):
  same_values_mask = df.apply(lambda row: all(value == '' or value == row.iloc[0] for value in row), axis=1)
  df.loc[same_values_mask, df.columns[1:]] = ''
  return df


def row_splitter(df):
    df['Marked'] = (df.apply(lambda row: any(bool(re.search(r'\$\s*\(\s*[\d,]+\s*\)|\$\s*[\d,]+|\(\s*[\d,]+\s*%\s*\)|[\d,]+\s*%',
                                                            str(cell).replace(" ", ""))) for cell in row), axis=1))
    first_true_index = df.index[df['Marked']].min()
    df=df.drop(columns="Marked")
    header_rows = df.iloc[:first_true_index, :]
    df_remaining = df.iloc[first_true_index:, :]
    return header_rows, df_remaining

In [ ]:
# df=assign_same_value(df)
# df=clean_repeating_rows(df)
header_rows,df_remaining=row_splitter(df)
header_rows

,1,2,3,4,5,6
0,Paid subscribers(2) at:,,,,% Change Better (Worse),% Change Better (Worse)
1,(in millions),"July 1, 2023","April 1, 2023","July 2, 2022","July 1, 2023 vs. April 1, 2023","July 1, 2023 vs. July 2, 2022"
2,Disney+,,,,,


In [ ]:
df_remaining=assign_same_value(df_remaining)
df_remaining=clean_repeating_rows(df_remaining)
df_remaining

,1,2,3,4,5,6
3,Domestic (U.S. and Canada),46.0,46.3,44.5,(1) %,3 %
4,International (excluding Disney+ Hotstar)(3),59.7,58.6,49.2,2 %,21 %
5,Disney+ Core(4),105.7,104.9,93.6,1 %,13 %
6,Disney+ Hotstar,40.4,52.9,58.4,(24) %,(31) %
7,ESPN+,25.2,25.3,22.8,— %,11 %
8,Hulu,,,,,
9,SVOD Only,44.0,43.7,42.2,1 %,4 %
10,Live TV + SVOD,4.3,4.4,4.0,(2) %,8 %
11,Total Hulu(4),48.3,48.2,46.2,— %,5 %


# Under Work


## Blank Rows

In [ ]:
import pandas as pd

def blank_rows_df(df):
  first_column=pd.DataFrame(df.iloc[:,0])
  empty_rows=first_column[first_column.iloc[:,0]== '']
  rows = []

  for index in empty_rows.index:
      if index==0:
        rows.append(index)
      elif index - 1 in empty_rows.index:
          rows.append(index)
      else:
        break
  return df.iloc[rows]

## is_Multi_Indexable

In [ ]:
def multi_indexer(df):
  heading_rows_df=blank_rows_df(df)
  num_rows=len(heading_rows_df)
  rows_list = [heading_rows_df.iloc[i].tolist() for i in range(num_rows)]
  tupled_indices = list(zip(*rows_list))
  level_names = [f"lvl-{i}" for i in range(num_rows)]
  multi_index_columns=pd.MultiIndex.from_tuples(tupled_indices, names=level_names)
  df.columns=multi_index_columns
  df=df.iloc[num_rows:]
  df=df.reset_index(drop=True)
  return df

In [ ]:
better_df=multi_indexer(df)
better_df

lvl-0                                                \
lvl-1                                                 
lvl-2                                                 
0                                      Net revenues   
1                                Operating expenses   
2                                  Operating income   
3                                  Operating margin   
4                       Other income (expense), net   
5                                Income tax expense   
6                                Effective tax rate   
7                                 Net income (loss)   
8               Net income (loss) per diluted share   
9      Net cash provided by operating activities(1)   

lvl-0                     Three Months Ended September 30,  \
lvl-1                                                 2023   
lvl-2 (In millions, except percentages and per share data)   
0                                                  $7418     
1                                                   6250     
2                                                  $1168     
3                                                    16%     
4                                                    $73     
5                                                    221     
6                                                    18%     
7                                                  $1020     
8                                                  $0.93     
9                                                  $1259     

lvl-0                                                       \
lvl-1                                                 2022   
lvl-2 (In millions, except percentages and per share data)   
0                                                  $6846     
1                                                   5728     
2                                                  $1118     
3                                                    16%     
4                                                   $460     
5                                                    248     
6                                                    16%     
7                                                  $1330     
8                                                  $1.15     
9                                                  $1755     

lvl-0                          Percent Increase/(Decrease)  \
lvl-1                          Percent Increase/(Decrease)   
lvl-2 (In millions, except percentages and per share data)   
0                                                     8%     
1                                                     9%     
2                                                     4%     
3                                                     **     
4                                                  (84)%     
5                                                  (11)%     
6                                                     **     
7                                                  (23)%     
8                                                  (19)%     
9                                                  (28)%     

lvl-0                      Nine Months Ended September 30,  \
lvl-1                                                 2023   
lvl-2 (In millions, except percentages and per share data)   
0                                                 $21745     
1                                                  18445     
2                                                  $3300     
3                                                    15%     
4                                                   $318     
5                                                    774     
6                                                    21%     
7                                                  $2844     
8                                                  $2.55     
9                                                  $2229     

lvl-0                                                   

## General Multi-Indexer

In [ ]:
df.iloc[:, 1]

0                      Three Months Ended September 30,
1                                                  2023
2     (In millions, except percentages and per share...
3                                                 $7418
4                                                  6250
5                                                 $1168
6                                                   16%
7                                                   $73
8                                                   221
9                                                   18%
10                                                $1020
11                                                $0.93
12                                                $1259
Name: 2, dtype: object

In [ ]:
def function_name(df):
  heading_rows_df=blank_rows_df(df)
    if len(temp_df)<=1:
      return df
    else:
      if condition-1:
        return multi_indexer(df)
      if condition-2:
        return row_merger(df)

In [ ]:
def is_multi_indexable(temp_df):
  if len(temp_df)<=1:
    return False
  else:

## Hide


In [ ]:
import pandas as pd
table_html = mdna_tables[index].html_tag.get_source_code()
df = pd.read_html(table_html)[0]
df.head(100)

In [ ]:
from bs4 import BeautifulSoup
table_html = mdna_tables[index].html_tag.get_source_code()
soup = BeautifulSoup(table_html, 'html.parser')
soup

## Preprocessing Functions

In [ ]:
def basic_preprocessing(soup):
  df = pd.read_html(str(soup))[0]
  df = df.dropna(how='all')
  df.columns = df.iloc[0]
  df = df[1:]
  df = df.dropna(axis=1, how='all')
  first_row = [col for col in df.columns]
  new_df=pd.DataFrame([first_row],columns=df.columns)
  df = pd.concat([new_df,df], ignore_index=True)
  df.columns = range(1, len(df.columns) + 1)
  df = df.fillna('')
  df=df.astype(str)
  return df

def remove_blank_columns(df):
    columns_to_remove = []
    for i in range(len(df.columns)):
        if i < len(df.columns)-1:
            col1 = df.columns[i]
            col2 = df.columns[i + 1]
            non_empty_rows = df[(df[col1] != '') & (df[col2] != '')]
            if (non_empty_rows[col1] == non_empty_rows[col2]).all():
                if (df[col1] == '').sum() >= (df[col2] == '').sum():
                    columns_to_remove.append(col1)
                else:
                    columns_to_remove.append(col2)
    df.drop(columns=columns_to_remove, inplace=True)
    return df


def merge_dollar_columns(df):
    for i, _ in enumerate(df.columns):
      if i < len(df.columns)-1:
        col1 = df.columns[i]
        col2 = df.columns[i + 1]

        # Ignore rows with '$' symbol
        non_dollar_rows = df[(df[col1] != '$') & (df[col2] != '$')]
        non_dollar_rows=non_dollar_rows[(non_dollar_rows[col1] != '') & (non_dollar_rows[col2] != '')]
        dollar_rows=df.drop(non_dollar_rows.index)
        dollar_rows_indices=dollar_rows.index

        # Check if remaining rows contain the same value
        if (non_dollar_rows[col1] == non_dollar_rows[col2]).all():
            df.loc[dollar_rows_indices, col1] = (df.loc[dollar_rows_indices, col1]
                                                 .str.cat(df.loc[dollar_rows_indices, col2], sep='', na_rep='')
                                                 .str.strip())
            df.drop(columns=[col2], inplace=True)
    return df


def merge_percentage_columns(df):
    for i, _ in enumerate(df.columns):
      if i < len(df.columns)-1:
        col1 = df.columns[i]
        col2 = df.columns[i + 1]

        # Ignore rows with '%' symbol
        non_dollar_rows = df[(df[col1] != '%') & (df[col2] != '%')]
        non_percentage_rows=non_dollar_rows[(non_dollar_rows[col1] != '') & (non_dollar_rows[col2] != '')]
        percentage_rows=df.drop(non_percentage_rows.index)
        percentage_rows_indices=percentage_rows.index

        # Check if remaining rows contain the same value
        if (non_percentage_rows[col1] == non_percentage_rows[col2]).all():
          df.loc[percentage_rows_indices, col1] = df.loc[percentage_rows_indices, col1].str.cat(df.loc[percentage_rows_indices, col2], sep='', na_rep='').str.strip()
          df.drop(columns=[col2], inplace=True)
    return df

## Parsed Tables

In [ ]:
df_cleaned=basic_preprocessing(soup)
df_cleaned=remove_blank_columns(df_cleaned)
df_cleaned=merge_dollar_columns(df_cleaned)
df_cleaned=merge_percentage_columns(df_cleaned)
df_cleaned.columns = range(1, len(df_cleaned.columns) + 1)
df_cleaned.head(100)

,1,2,3,4,5
0,Gross margin percentage:,,,,
1,Products,35.4%,34.5%,36.5%,36.8%
2,Services,70.5%,71.5%,70.8%,72.2%
3,Total gross margin percentage,44.5%,43.3%,43.8%,43.6%


In [ ]:
json_data = df_cleaned.to_json(orient='records')
json_data

'[{"1":"","2":"Three months ended June 30,","3":"Three months ended June 30,","4":"Six months ended June 30,","5":"Six months ended June 30,"},{"1":"(in\\u00a0millions)","2":"2023","3":"2022","4":"2023","5":"2022"},{"1":"Interest expense","2":"$552","3":"$556","4":"$1105","5":"$1104"},{"1":"Interest income","2":"(98)","3":"(24)","4":"(197)","5":"(33)"},{"1":"Interest expense, net","2":"$454","3":"$532","4":"$908","5":"$1071"},{"1":"Net foreign exchange loss","2":"$37","3":"$47","4":"$72","5":"$72"},{"1":"Other expense, net","2":"1412","3":"1533","4":"3216","5":"757"}]'

In [ ]:
# GPT3_ENCODING = "cl100k_base"


# def num_tokens_from_string(string: str, encoding_name: str) -> int:
#     """Returns the number of tokens in a text string."""
#     encoding = tiktoken.get_encoding(encoding_name)
#     num_tokens = len(encoding.encode(string))
#     return num_tokens


# num_tokens_from_string(markdown, GPT4_ENCODING)

In [ ]:
from sec_parser.semantic_elements.table_element import TableElement
from sec_parser.processing_engine.html_tag import HtmlTag
from bs4 import BeautifulSoup

# # Let's assume you have some HTML for a table
# html = """
# <table>
#     <tr>
#         <td>1</td>
#         <td>2</td>
#     </tr>
#     <tr>
#         <td>3</td>
#         <td>4</td>
#     </tr>
# </table>
# """

# # Parse the HTML using BeautifulSoup
# soup = BeautifulSoup(html, "html.parser")

# # Create an HtmlTag from the BeautifulSoup object
# html_tag = HtmlTag(soup.table)

# # Create a TableElement from the HtmlTag
# table_element = TableElement(html_tag)
# table_element



mdna_tables[0].to_dict()

{'cls_name': 'TableElement',
 'tag_name': 'div',
 'text_preview': 'Production LocationV...[339]...adsterIn development',
 'html_preview': '<div style="margin-t...[8790]...></tr></table></div>',
 'html_hash': '3682c14c',
 'metrics': {'rows': 8, 'numbers': 2}}

In [ ]:
!git clone https://github.com/alphanome-ai/sec-parser
%cd "/content/sec-parser"

In [ ]:
!pip install poetry
!poetry export -f requirements.txt --output requirements.txt
!pip install -r "/content/sec-parser/requirements.txt"

In [ ]:
from bs4 import BeautifulSoup

def colorize_html_background_inline(input_html):
    soup = BeautifulSoup(input_html, 'html.parser')

    # Define background colors
    bg_colors = {
        'h1': 'background-color: #ff0000;',  # Red
        'h2': 'background-color: #ff0000;',  # Red
        'p': 'background-color: #0000ff;',  # Blue
        'table': 'background-color: #00ff00;',  # Green
        # Add more mappings for other semantic elements as needed
    }

    # Apply background colors to semantic elements
    for element, bg_style in bg_colors.items():
        for tag in soup.find_all(element):
            if 'style' in tag.attrs:
                tag['style'] += f' {bg_style}'
            else:
                tag['style'] = bg_style

    # Return the colored HTML
    return str(soup)

In [ ]:
html_input = """
<html>
  <head>
    <title>Sample HTML</title>
  </head>
  <body>
    <h1>This is a Heading 1</h1>
    <p>Lorem ipsum dolor sit amet...</p>
    <h2>This is a Heading 2</h2>
    <table border="1">
      <tr>
        <td>Table Cell 1</td>
        <td>Table Cell 2</td>
      </tr>
    </table>
  </body>
</html>
"""

In [ ]:
colored_html = colorize_html_background_inline(html_input)
print(colored_html)


<html>
<head>
<title>Sample HTML</title>
</head>
<body>
<h1 style="background-color: #ff0000;">This is a Heading 1</h1>
<p style="background-color: #0000ff;">Lorem ipsum dolor sit amet...</p>
<h2 style="background-color: #ff0000;">This is a Heading 2</h2>
<table border="1" style="background-color: #00ff00;">
<tr>
<td>Table Cell 1</td>
<td>Table Cell 2</td>
</tr>
</table>
</body>
</html>



##

In [156]:
TABLE_10Q_CP_0000016875_20_000032__001 = """
<table cellpadding="0" cellspacing="0" style="font-family:Times New Roman;font-size:10pt;width:100%;border-collapse:collapse;text-align:left;"><tbody><tr><td colspan="13"></td></tr><tr><td style="width:60%;"></td><td style="width:1%;"></td><td style="width:8%;"></td><td style="width:1%;"></td><td style="width:1%;"></td><td style="width:8%;"></td><td style="width:1%;"></td><td style="width:1%;"></td><td style="width:8%;"></td><td style="width:1%;"></td><td style="width:1%;"></td><td style="width:8%;"></td><td style="width:1%;"></td></tr><tr><td style="vertical-align:bottom;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="overflow:hidden;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;">&nbsp;</span></div></td><td colspan="6" style="vertical-align:bottom;border-bottom:1px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:center;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;">For the three months ended September 30</span></div></td><td colspan="6" style="vertical-align:bottom;border-bottom:1px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:center;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;">For the nine months ended September 30</span></div></td></tr><tr><td style="vertical-align:top;border-bottom:2px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">(in millions of Canadian dollars)</span></div></td><td colspan="3" style="vertical-align:bottom;border-bottom:2px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="text-align:center;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;">2020</span></div></td><td colspan="3" style="vertical-align:bottom;border-bottom:2px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="text-align:center;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">2019</span></div></td><td colspan="3" style="vertical-align:bottom;border-bottom:2px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="text-align:center;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;">2020</span></div></td><td colspan="3" style="vertical-align:bottom;border-bottom:2px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="text-align:center;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">2019</span></div></td></tr><tr><td style="vertical-align:middle;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;border-top:2px solid #000000;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">Net income</span></div></td><td style="vertical-align:bottom;padding-left:2px;padding-top:2px;padding-bottom:2px;background-color:#cceeff;border-top:2px solid #000000;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;">$</span></div></td><td style="vertical-align:bottom;background-color:#cceeff;padding-top:2px;padding-bottom:2px;border-top:2px solid #000000;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e880-wk-Fact-3D6A129E4F7E11A9FC47268C162C2C40" name="us-gaap:NetIncomeLoss" contextref="FD2020Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">598</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;background-color:#cceeff;border-top:2px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td style="vertical-align:bottom;padding-left:2px;padding-top:2px;padding-bottom:2px;background-color:#cceeff;border-top:2px solid #000000;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">$</span></div></td><td style="vertical-align:bottom;background-color:#cceeff;padding-top:2px;padding-bottom:2px;border-top:2px solid #000000;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e894-wk-Fact-CB1B6FD42F35C73D08B9268C164DF9E2" name="us-gaap:NetIncomeLoss" contextref="FD2019Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">618</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;background-color:#cceeff;border-top:2px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td style="vertical-align:bottom;padding-left:2px;padding-top:2px;padding-bottom:2px;background-color:#cceeff;border-top:2px solid #000000;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;">$</span></div></td><td style="vertical-align:bottom;background-color:#cceeff;padding-top:2px;padding-bottom:2px;border-top:2px solid #000000;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e908-wk-Fact-760561D0617E2A60BA84268C16359E69" name="us-gaap:NetIncomeLoss" contextref="FD2020Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">1,642</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;background-color:#cceeff;border-top:2px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td style="vertical-align:bottom;padding-left:2px;padding-top:2px;padding-bottom:2px;background-color:#cceeff;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">$</span></div></td><td style="vertical-align:bottom;background-color:#cceeff;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e923-wk-Fact-96A31269D8012275E97E268C1690257F" name="us-gaap:NetIncomeLoss" contextref="FD2019Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">1,776</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;background-color:#cceeff;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td></tr><tr><td style="vertical-align:middle;padding-left:12px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">Net gain (loss) in foreign currency translation adjustments, net of hedging activities</span></div></td><td colspan="2" style="vertical-align:bottom;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e938-wk-Fact-8EF64D4AB5FB77A51497268C162CC754" name="us-gaap:OtherComprehensiveIncomeForeignCurrencyTransactionAndTranslationAdjustmentBeforeTaxPortionAttributableToParent" contextref="FD2020Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">16</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span>(<ix:nonfraction id="d742040e952-wk-Fact-82F9DCF8DF77557A88DE268C162CD183" name="us-gaap:OtherComprehensiveIncomeForeignCurrencyTransactionAndTranslationAdjustmentBeforeTaxPortionAttributableToParent" contextref="FD2019Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" sign="-" format="ixt:numdotdecimal">8</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;padding-right:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">)</span></div></td><td colspan="2" style="vertical-align:bottom;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span>(<ix:nonfraction id="d742040e967-wk-Fact-11E0588AB5CB01AF405B268C1635A90A" name="us-gaap:OtherComprehensiveIncomeForeignCurrencyTransactionAndTranslationAdjustmentBeforeTaxPortionAttributableToParent" contextref="FD2020Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" sign="-" format="ixt:numdotdecimal">18</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;padding-right:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;">)</span></div></td><td colspan="2" style="vertical-align:bottom;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e982-wk-Fact-4F15035DBBF815457F23268C16230BF7" name="us-gaap:OtherComprehensiveIncomeForeignCurrencyTransactionAndTranslationAdjustmentBeforeTaxPortionAttributableToParent" contextref="FD2019Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">23</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td></tr><tr><td style="vertical-align:middle;background-color:#cceeff;padding-left:12px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">Change in derivatives designated as cash flow hedges</span></div></td><td colspan="2" style="vertical-align:bottom;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e1003-wk-Fact-C4BDAD8CAFC63B5D2BE8268C162C8C18" name="us-gaap:OtherComprehensiveIncomeLossCashFlowHedgeGainLossAfterReclassificationBeforeTax" contextref="FD2020Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">3</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;background-color:#cceeff;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e1017-wk-Fact-AEDCEDD9EDBC1058BD93268C1635E834" name="us-gaap:OtherComprehensiveIncomeLossCashFlowHedgeGainLossAfterReclassificationBeforeTax" contextref="FD2019Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">2</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;background-color:#cceeff;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e1031-wk-Fact-3CBC8509783503A8ED8F268C1661B275" name="us-gaap:OtherComprehensiveIncomeLossCashFlowHedgeGainLossAfterReclassificationBeforeTax" contextref="FD2020Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">6</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;background-color:#cceeff;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e1045-wk-Fact-0E1C8BCC58832708BB1B268C168324E2" name="us-gaap:OtherComprehensiveIncomeLossCashFlowHedgeGainLossAfterReclassificationBeforeTax" contextref="FD2019Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">8</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;background-color:#cceeff;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td></tr><tr><td style="vertical-align:middle;border-bottom:1px solid #000000;padding-left:12px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">Change in pension and post-retirement defined benefit plans</span></div></td><td colspan="2" style="vertical-align:bottom;border-bottom:1px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e1066-wk-Fact-C6EB30760AAEB1B1BA11268C163D70B0" name="us-gaap:OtherComprehensiveIncomeDefinedBenefitPlansAdjustmentBeforeTaxPortionAttributableToParent" contextref="FD2020Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" sign="-" format="ixt:numdotdecimal">44</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:1px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;border-bottom:1px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e1080-wk-Fact-B4FCF227E9E8F55F0DB4268C162CA9FD" name="us-gaap:OtherComprehensiveIncomeDefinedBenefitPlansAdjustmentBeforeTaxPortionAttributableToParent" contextref="FD2019Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" sign="-" format="ixt:numdotdecimal">20</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:1px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;border-bottom:1px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e1094-wk-Fact-FDFE3DB054D34AD61F15268C165548D9" name="us-gaap:OtherComprehensiveIncomeDefinedBenefitPlansAdjustmentBeforeTaxPortionAttributableToParent" contextref="FD2020Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" sign="-" format="ixt:numdotdecimal">134</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:1px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;border-bottom:1px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e1108-wk-Fact-DCB2273CAB14365EA2C2268C162CD5FA" name="us-gaap:OtherComprehensiveIncomeDefinedBenefitPlansAdjustmentBeforeTaxPortionAttributableToParent" contextref="FD2019Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" sign="-" format="ixt:numdotdecimal">61</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:1px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td></tr><tr><td style="vertical-align:middle;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;border-top:1px solid #000000;"><div style="padding-bottom:6px;padding-top:6px;text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">Other comprehensive income before income taxes</span></div></td><td colspan="2" style="vertical-align:bottom;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e1129-wk-Fact-E719CB56721AF5D2E444268C169025FB" name="us-gaap:OtherComprehensiveIncomeLossBeforeTaxPortionAttributableToParent" contextref="FD2020Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">63</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;background-color:#cceeff;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;border-top:1px solid #000000;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e1143-wk-Fact-D70B0E5857C2E151A05D268C162C20F5" name="us-gaap:OtherComprehensiveIncomeLossBeforeTaxPortionAttributableToParent" contextref="FD2019Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">14</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;background-color:#cceeff;border-top:1px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e1157-wk-Fact-6CB231651BEF5D665250268C1623A861" name="us-gaap:OtherComprehensiveIncomeLossBeforeTaxPortionAttributableToParent" contextref="FD2020Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">122</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;background-color:#cceeff;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;border-top:1px solid #000000;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e1171-wk-Fact-4D31160244DEB4CA615A268C163DFF56" name="us-gaap:OtherComprehensiveIncomeLossBeforeTaxPortionAttributableToParent" contextref="FD2019Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">92</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;background-color:#cceeff;border-top:1px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td></tr><tr><td style="vertical-align:middle;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">Income tax (expense) recovery on above items</span></div></td><td colspan="2" style="vertical-align:bottom;border-bottom:1px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span>(<ix:nonfraction id="d742040e1192-wk-Fact-7E556294B8C7A840EB0E268C163D9789" name="us-gaap:OtherComprehensiveIncomeLossTaxPortionAttributableToParent1" contextref="FD2020Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">29</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:1px solid #000000;padding-right:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;">)</span></div></td><td colspan="2" style="vertical-align:bottom;border-bottom:1px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e1207-wk-Fact-BCB65CB569842B29F18F268C16834193" name="us-gaap:OtherComprehensiveIncomeLossTaxPortionAttributableToParent1" contextref="FD2019Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" sign="-" format="ixt:numdotdecimal">3</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:1px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;border-bottom:1px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span>(<ix:nonfraction id="d742040e1221-wk-Fact-2B94915C76EFABCE83C9268C1683EEDE" name="us-gaap:OtherComprehensiveIncomeLossTaxPortionAttributableToParent1" contextref="FD2020Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">16</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:1px solid #000000;padding-right:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;">)</span></div></td><td colspan="2" style="vertical-align:bottom;border-bottom:1px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span>(<ix:nonfraction id="d742040e1236-wk-Fact-2D003962DC30F86220AC268C165584F7" name="us-gaap:OtherComprehensiveIncomeLossTaxPortionAttributableToParent1" contextref="FD2019Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">41</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:1px solid #000000;padding-right:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">)</span></div></td></tr><tr><td style="vertical-align:middle;border-bottom:1px solid #000000;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;border-top:1px solid #000000;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">Other comprehensive income (Note 7)</span></div></td><td colspan="2" style="vertical-align:bottom;border-bottom:1px solid #000000;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e1258-wk-Fact-DF2AB8FCA3E589FE8EF4268C168EA36E" name="us-gaap:OtherComprehensiveIncomeLossNetOfTaxPortionAttributableToParent" contextref="FD2020Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">34</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:1px solid #000000;background-color:#cceeff;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;border-bottom:1px solid #000000;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;border-top:1px solid #000000;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e1272-wk-Fact-4293B31D3A08549A2528268C16900287" name="us-gaap:OtherComprehensiveIncomeLossNetOfTaxPortionAttributableToParent" contextref="FD2019Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">17</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:1px solid #000000;background-color:#cceeff;border-top:1px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;border-bottom:1px solid #000000;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e1286-wk-Fact-D988D7EF662E68EDC196268C1690CAE4" name="us-gaap:OtherComprehensiveIncomeLossNetOfTaxPortionAttributableToParent" contextref="FD2020Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">106</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:1px solid #000000;background-color:#cceeff;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td colspan="2" style="vertical-align:bottom;border-bottom:1px solid #000000;background-color:#cceeff;padding-left:2px;padding-top:2px;padding-bottom:2px;border-top:1px solid #000000;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e1300-wk-Fact-5FF006F7DC9770AB9B0F268C16832980" name="us-gaap:OtherComprehensiveIncomeLossNetOfTaxPortionAttributableToParent" contextref="FD2019Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">51</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:1px solid #000000;background-color:#cceeff;border-top:1px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td></tr><tr><td style="vertical-align:middle;border-bottom:2px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;">Comprehensive income</span></div></td><td style="vertical-align:bottom;border-bottom:2px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;">$</span></div></td><td style="vertical-align:bottom;border-bottom:2px solid #000000;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e1326-wk-Fact-6DA351E677DC0D5FD391268C168B94DB" name="us-gaap:ComprehensiveIncomeNetOfTax" contextref="FD2020Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">632</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:2px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td style="vertical-align:bottom;border-bottom:2px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;border-top:1px solid #000000;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">$</span></div></td><td style="vertical-align:bottom;border-bottom:2px solid #000000;padding-top:2px;padding-bottom:2px;border-top:1px solid #000000;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e1340-wk-Fact-0A433E51D426926BCEE0268C1635D4C8" name="us-gaap:ComprehensiveIncomeNetOfTax" contextref="FD2019Q3QTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">635</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:2px solid #000000;border-top:1px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td style="vertical-align:bottom;border-bottom:2px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;">$</span></div></td><td style="vertical-align:bottom;border-bottom:2px solid #000000;padding-top:2px;padding-bottom:2px;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;font-weight:bold;"><span><ix:nonfraction id="d742040e1354-wk-Fact-D8DE420A377FDC9771FD268C1690DDF9" name="us-gaap:ComprehensiveIncomeNetOfTax" contextref="FD2020Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">1,748</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:2px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td><td style="vertical-align:bottom;border-bottom:2px solid #000000;padding-left:2px;padding-top:2px;padding-bottom:2px;border-top:1px solid #000000;"><div style="text-align:left;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;">$</span></div></td><td style="vertical-align:bottom;border-bottom:2px solid #000000;padding-top:2px;padding-bottom:2px;border-top:1px solid #000000;"><div style="text-align:right;font-size:9pt;"><span style="font-family:Arial;font-size:9pt;"><span><ix:nonfraction id="d742040e1369-wk-Fact-3000F1CD0547D05A4C25268C162CA6F5" name="us-gaap:ComprehensiveIncomeNetOfTax" contextref="FD2019Q3YTD" unitref="iso4217_CAD" decimals="-6" scale="6" format="ixt:numdotdecimal">1,827</ix:nonfraction></span></span></div></td><td style="vertical-align:bottom;border-bottom:2px solid #000000;border-top:1px solid #000000;"><div style="text-align:left;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;"><br></span></div></td></tr></tbody></table>
"""
